<a href="https://colab.research.google.com/github/inoueshinichi/3DGeometry/blob/main/PytorchTutorial_p2ch3_tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

#Pytorchにおけるテンソルの扱い方

多次元配列としてのテンソル

In [4]:
# Python list -> Pytorch tensor
a = [1.0, 2.0, 1.0]
a

[1.0, 2.0, 1.0]

In [5]:
a[2] = 3.0
a

[1.0, 2.0, 3.0]

In [6]:
# tensor
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [7]:
a[1]

tensor(1.)

In [8]:
float(a[0])

1.0

In [9]:
a[2] = 2.0
a

tensor([1., 1., 2.])

座標(4,1),(5,3),(2,1)を頂点とする2次元の三角形

In [10]:
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

In [11]:
points = torch.tensor([4.,1.,5.,3.,2.,1.])
points

tensor([4., 1., 5., 3., 2., 1.])

In [12]:
# 最初の点
float(points[0]), float(points[1])

(4.0, 1.0)

２次元テンソル

In [13]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [14]:
# 形状
points.shape

torch.Size([3, 2])

In [15]:
# zeros
points = torch.zeros(3,2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [16]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [17]:
points[0,1]

tensor(1.)

In [18]:
points[0]

tensor([4., 1.])

テンソルの一部指定や取り出し(Indexing)

In [19]:
some_list = list(range(6))
print(some_list[:]) # 全要素の指定
print(some_list[1:4]) # 1から4の手前までを指定
print(some_list[1:] ) # 1以降を指定
print(some_list[:4] ) # 最初から4の手前までを指定
print(some_list[:-1]) # 最初から最後の1つ手前までを指定
print(some_list[1:4:2]) # 1から4の手前まで2つ置きに指定

[0, 1, 2, 3, 4, 5]
[1, 2, 3]
[1, 2, 3, 4, 5]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[1, 3]


In [20]:
print(points[1:]   )
print(points[1:, :])
print(points[1:, 0])
print(points[None] )# 新たな次元を追加。unsqueezeと同じ働き

tensor([[5., 3.],
        [2., 1.]])
tensor([[5., 3.],
        [2., 1.]])
tensor([5., 2.])
tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])


#名前付きテンソル(named tensor)

In [21]:
img_t = torch.randn(3,5,5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [22]:
batch_t = torch.randn(2,3,5,5) # shape [batch, channels, rows, columns]

In [23]:
# 画像データは常に-3次元目にあることを想定した処理
img_gray_naive = img_t.mean(-3) # 末尾から3つ遡った次元で平均を取る
print(img_gray_naive)
print(img_gray_naive.shape)
batch_gray_naive = batch_t.mean(-3)
print(batch_gray_naive)
print(batch_gray_naive.shape)

tensor([[-0.9043,  0.6406, -0.8907, -0.1443,  0.5758],
        [ 0.0559, -0.2708, -0.8359,  0.2216, -1.4901],
        [ 0.6648, -0.6183, -0.1255,  1.0274, -0.1120],
        [ 0.2000,  0.8271, -0.0144,  0.3996,  0.3855],
        [ 0.8497,  0.1914, -0.3041, -0.1335,  1.0073]])
torch.Size([5, 5])
tensor([[[ 0.0398,  0.4369, -0.5379, -0.8074,  0.7055],
         [-1.1460, -0.5137, -0.5970, -0.2656,  0.2752],
         [ 0.8712,  0.0821,  0.1365, -0.2216,  0.7509],
         [-0.1791, -0.4473,  0.4871,  0.5644,  0.3505],
         [ 0.4228, -0.6932, -0.6435, -0.0561, -0.2631]],

        [[-0.4402,  0.1215, -1.1329, -0.4171,  0.5484],
         [-0.8717, -0.7567, -0.7743,  0.4987,  0.5941],
         [-0.0238, -0.2864,  0.5216,  0.5810,  0.0616],
         [-0.5876, -1.0920, -0.2794, -0.0530,  0.3462],
         [ 0.5591,  0.2228,  0.0349, -0.2871,  0.0204]]])
torch.Size([2, 5, 5])


#Broadcasting

In [24]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1) # (3,) -> (3,1,1)
print(unsqueezed_weights.shape)
img_weights = (img_t * unsqueezed_weights) # (3,5,5) * (3,1,1) -> (3,5,5)
print(img_weights.shape)
batch_weights = (batch_t * unsqueezed_weights) # (2,3,5,5) * (3,1,1) -> (2,3,5,5)
print(batch_weights.shape)

img_gray_weighted = img_weights.sum(-3) # channelsの軸で加算
print(img_gray_weighted)

batch_gray_weighted = batch_weights.sum(-3) # channelsの軸で加算
print(batch_gray_weighted)

torch.Size([3, 1, 1])
torch.Size([3, 5, 5])
torch.Size([2, 3, 5, 5])
tensor([[-1.1595,  0.7353, -0.9738, -0.8590,  0.3829],
        [-0.3012, -1.9156, -1.0862,  0.2704, -1.8622],
        [ 0.9187, -0.5816, -0.6028,  0.1677, -0.6534],
        [ 0.6473,  1.1677,  0.2565, -0.0500,  0.2756],
        [ 0.3459,  0.3953, -0.2630,  0.5067,  1.3309]])
tensor([[[ 0.4928, -0.0229, -1.0054, -0.5014,  1.0885],
         [-1.1989, -0.2975, -0.7805, -1.6274,  0.2685],
         [-0.3824,  0.4849,  0.0514, -1.0804, -0.1603],
         [-0.4966, -0.4541,  0.6223,  0.6775,  0.7976],
         [-0.7178, -0.6630, -0.3689, -0.4452,  0.3405]],

        [[-0.1935, -0.6730, -1.7690, -0.5535,  1.2103],
         [-1.3087, -0.2260, -0.4708,  0.0695,  0.9731],
         [-0.2647, -0.3851, -0.3518,  0.0036,  0.3455],
         [-0.6623, -0.9032, -0.0689,  0.1660, -0.4213],
         [ 0.6108, -0.2813,  0.5576, -0.6227,  0.5457]]])


Pytorchの関数einsum(Numpyからの転用)で、上記の計算結果のテンソルの次元にインデックス名を与えるインデックス付けをする

In [25]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
print(img_gray_weighted_fancy)

tensor([[-1.1595,  0.7353, -0.9738, -0.8590,  0.3829],
        [-0.3012, -1.9156, -1.0862,  0.2704, -1.8622],
        [ 0.9187, -0.5816, -0.6028,  0.1677, -0.6534],
        [ 0.6473,  1.1677,  0.2565, -0.0500,  0.2756],
        [ 0.3459,  0.3953, -0.2630,  0.5067,  1.3309]])


#名前付きテンソル (>pytorch1.3)  
https://pytorch.org/tutorials/intermediate/named_tensor_tutorial.html  
https://pytorch.org/docs/stable/named_tensor.html

In [26]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1408.)
  """Entry point for launching an IPython kernel.


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [27]:
img_named = torch.randn(3,5,5, names=['C','H','W'])
img_named

tensor([[[ 0.1411, -1.3354, -2.9340,  0.1141, -1.2072],
         [-0.3008,  0.1427, -0.9592, -0.4919, -2.1429],
         [ 0.9488, -0.5684, -0.0646,  0.6647, -2.7836],
         [-1.4483,  0.9089,  0.9494,  0.0266, -0.9221],
         [ 0.7034, -0.3659, -0.1965,  1.7250,  0.3154]],

        [[-0.0217,  0.3441,  0.2271, -0.4597, -0.6183],
         [ 0.2461,  1.2119, -0.8368,  1.2277, -0.4297],
         [-2.2121, -0.3780,  0.9838, -1.0895, -0.6378],
         [ 0.0221, -1.7753, -0.7490,  0.2781, -0.9621],
         [-0.4223, -1.1036,  0.8398,  1.4549, -0.2835]],

        [[-0.3767, -0.0306, -0.0894, -0.1965, -0.9713],
         [ 0.2790, -0.2523,  1.0669, -0.2985, -0.7259],
         [ 0.7346, -0.0845,  1.0757,  0.6367, -0.1943],
         [-0.8614,  0.5338,  0.3940, -2.0565,  1.1062],
         [ 0.4562,  0.0144, -0.6411,  2.3902, -1.4256]]],
       names=('C', 'H', 'W'))

すでにテンソルが存在し、名前を追加したい場合、そのテンソルに対してrefine_namesメソッドを使用する.  
renameメソッドを使用すると、既存の名前を上書きしたり、(Noneを指定して)削除出来る

In [28]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named: ', img_named.shape, img_named.names)
print('batch named: ', batch_named.shape, batch_named.names)

img named:  torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named:  torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


名前付きテンソルのブロードキャストには, aline_asメソッドを使って次元数の増減で演算操作に整合性をもたせる

In [29]:
weights_aligned = weights_named.align_as(img_named) # 名前が継承される
weights_named.shape, weights_aligned.shape, weights_aligned.names

(torch.Size([3]), torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

関数sumのように次元の引数を持つ関数は、次元の名前でも演算する次元を指定可能

In [30]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

異なる名前の次元を結合しようとするとエラーが発生する

In [31]:
gray_named = (img_named[..., :3] * weights_named).sum('channels')

RuntimeError: ignored

名前付きテンソルを操作する関数の外で使いたい場合、名前をNoneにリネームして名前を削除する必要がある

In [32]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

#テンソルのdtype属性の管理

In [33]:
double_points = torch.ones(10, 2, dtype=torch.double)
print(double_points)
short_points = torch.tensor([[1,2],[3,4]], dtype=torch.short)
print(short_points)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], dtype=torch.float64)
tensor([[1, 2],
        [3, 4]], dtype=torch.int16)


In [34]:
short_points.dtype

torch.int16

In [35]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

to関数を使用することもできる(便利)

In [36]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

演算で入力型が混合する場合、入力は自動的に大きい方の型に変換されます。
したがって、３２bitで計算をしたい場合、すべての入力が最大でも32bitであることを確認する必要がある

In [37]:
points_64 = torch.rand(5, dtype=torch.double) # randは0-1の乱数を生成
print("points_64", points_64)

points_short = points_64.to(torch.short)
print("points_short", points_short)

points_64 * points_short # PyTorch 1.3以降で動作する -> 自動で範囲の広い型にキャストされる

points_64 tensor([0.4172, 0.4367, 0.4856, 0.4751, 0.9276], dtype=torch.float64)
points_short tensor([0, 0, 0, 0, 0], dtype=torch.int16)


tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

#テンソルに対するAPI（演算操作)

演算操作の大部分はtorchモジュールで利用可能

In [38]:
# transpose
a = torch.ones(3,2)
print('a', a)
a_t = torch.transpose(a, 0, 1)
print(a.shape, a_t.shape)
print('a_t', a_t)

a tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
torch.Size([3, 2]) torch.Size([2, 3])
a_t tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [39]:
# テンソルオブジェクト自体の関数としても書ける
a = torch.ones(3,2)
a_t = a.transpose(0, 1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

#テンソルの格納状態

ストレージへのインデックス化

In [40]:
# 与えられたテンソルのストレージは、.storageプロパティを使ってアクセス出来る
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(points)
points.storage()

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [41]:
# 手動でストレージにインデックスしてアクセス出来る
points_storage = points.storage()
points_storage[0], points_storage[1]

(4.0, 1.0)

In [42]:
# ストレージの値を変更すると、それを参照しているテンソルの内容も変更される
points.storage()[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

保存された値の変更, インプレース操作 -> 〇〇_()

In [43]:
a = torch.ones(3,2)

In [44]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

#テンソルのメタ情報(サイズ、オフセット、ストライド)

テンソルとストレージの表示関係

In [45]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
print(second_point.storage_offset())
third_point = points[2]
print(third_point.storage_offset())

2
4


In [46]:
second_point.size()

torch.Size([2])

In [47]:
print(second_point.shape)
print(second_point)

torch.Size([2])
tensor([5., 3.])


In [48]:
points.stride()

(2, 1)

In [49]:
second_point.shape

torch.Size([2])

In [50]:
second_point.stride()

(1,)

(i,j)アクセス -> storage_offset + stride[0] * i + stride[1] * j  
通常storage_offsetはゼロ

In [51]:
# テンソルが、より大きなテンソルを保持するために作成されたストレージのビューである場合、オフセットが正の値になる
second_point = points[1]
second_point.size()

torch.Size([2])

In [52]:
second_point.storage_offset()

2

In [53]:
second_point.stride()

(1,)

サブテンソルを変更すると元のテンソルに副作用がでることを意味する

In [54]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
print("second_point", second_point)
second_point[0] = 10.0
points # 副作用

second_point tensor([5., 3.])


tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

クローン(コピーによる)サブテンソルの作成

In [55]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
clone_second_point = points[1].clone()
second_point[0] = 10.0
points # 副作用なし

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

コピーせずに転置する

In [56]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [57]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

転置前と転置後のストレージが同じ場所を指している(共有している)状態のチェックセル

In [58]:
id(points.storage()) == id(points_t.storage())

True

shapeやstrideが異なるだけ

In [59]:
points.stride()

(2, 1)

In [60]:
points_t.stride()

(1, 2)

転置しても新しいメモリが割り当てられるわけでなない! stride(3,1) - .T -> stride(1,3) でストライドも転置する


In [61]:
ori = torch.tensor([[3,1,2],[4,1,1]], dtype=torch.int32)
print("ori", ori)
print("ori.stride", ori.stride())

ori tensor([[3, 1, 2],
        [4, 1, 1]], dtype=torch.int32)
ori.stride (3, 1)


In [62]:
ori_t = ori.T
print("ori.T", ori_t)
print("ori.T.stride", ori_t.stride()) # ストライドも転置している -> 新しいストレージが割り当てられるわけでない

ori.T tensor([[3, 4],
        [1, 1],
        [2, 1]], dtype=torch.int32)
ori.T.stride (1, 3)


### 高次元での転置

多次元配列を転置するには、形状とストライドの反転を行う2次元を指定する

In [63]:
some_t = torch.ones(3,4,5)
transpose_t = some_t.transpose(0,2)
print("some_t.shape", some_t.shape)
print("transpose_t.shape", transpose_t.shape)

some_t.shape torch.Size([3, 4, 5])
transpose_t.shape torch.Size([5, 4, 3])


In [64]:
print("some_t.stride", some_t.stride())
print("transpose_t.stride", transpose_t.stride())

some_t.stride (20, 5, 1)
transpose_t.stride (1, 5, 20)


### 連続配置したテンソル

In [65]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])

In [66]:
points.is_contiguous()

True

In [67]:
points_t = torch.transpose(points, 0, 1)
print(points_t)

tensor([[4., 5., 2.],
        [1., 3., 1.]])


In [68]:
points_t.is_contiguous()

False

contiguous関数を用いて、非連続なテンソルから新しい連続テンソルを取得できる

In [69]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(points)
points_t = points.t()
print(points_t)

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
tensor([[4., 5., 2.],
        [1., 3., 1.]])


In [70]:
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [71]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [72]:
points.stride()

(2, 1)

In [73]:
points_t.stride()

(1, 2)

In [74]:
points.is_contiguous()

True

In [75]:
points_t.is_contiguous()

False

In [76]:
points.contiguous()

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [77]:
points_t_cont = points_t.contiguous() # 新しいメモリが連続したテンソルを生成している

In [78]:
points_t.is_contiguous()

False

In [79]:
print(points_t_cont)
print(points_t_cont.is_contiguous())

tensor([[4., 5., 2.],
        [1., 3., 1.]])
True


In [80]:
print(id(points_t_cont) == id(points)) # もはや、同じメモリ領域を指していない.

False


In [81]:
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [82]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]

### テンソルのデバイス属性の管理

In [83]:
# GPU上にテンソルを配置する
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')

In [84]:
# CPUで作成したテンソルをto関数を使ってGPUにコピーする
points_gpu_to = points.to(device='cuda')

In [85]:
points_gpu_to

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]], device='cuda:0')

In [87]:
# GPUが複数台存在する場合, 整数で指定できる
points_gpu = points.to(device='cuda:0')

In [88]:
points = 2 * points # CPUでの掛け算
points_gpu = 2 * points.to(device='cuda') # GPUでの掛け算

In [89]:
points_gpu = points_gpu + 4 # 計算結果もGPU上にある

In [90]:
# GPU->CPU
points_cpu = points_gpu.to(device='cuda')

In [91]:
# cpu/cuda
points_gpu = points.cuda() # GPUの１個目に自動配置
points_gpu = points.cuda(0)
points_cpu = points_gpu.cpu()

### Numpyとの相互運用性

In [93]:
points = torch.ones(3,4)
points_np = points.numpy() # テンソルがCPU上ならメモリコピーは発生しない!
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [94]:
points = torch.from_numpy(points_np)

注意) Pytorchの型デフォルトはfloat32だが, numpyはfloat64なので、numpyからtorchに変換後にdtypeがtorch.floatであることを確認すべき

In [95]:
points.dtype

torch.float32

In [97]:
points_np.dtype

dtype('float32')

In [98]:
points_np_ori = np.array([3.0, 4.0])
points_np_ori.dtype

dtype('float64')

### テンソルの保存とロード(Serializing)

In [106]:
torch.save(points, "/content/ourpoints.t")

In [103]:
!pwd

/content


In [104]:
!cwd

/bin/bash: cwd: command not found


In [105]:
!ls -a

.  ..  .config	sample_data


In [107]:
with open("/content/ourpoints2.t", 'wb') as f:
  torch.save(points, f)

In [108]:
points_load = torch.load('/content/ourpoints.t')

In [109]:
with open('/content/ourpoints2.t', 'rb') as f:
  points_load2 = torch.load(f)

### h5pyによるHDF5での保存

In [110]:
import h5py

In [111]:
f = h5py.File('/content/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy()) # numpyに変換している
f.close()

In [122]:
# HDF5はkey-value辞書形式で構成された多次元配列を表現するためのフォーマット
# ディクス上のデータセットにインデックスをつけて、関心のある要素だけにアクセスできる(メモリに配置できる)
f = h5py.File('/content/ourpoints.hdf5', 'r')
dset = f['coords'] # key: coords
print('dset', dset)
last_points = dset[-3:]
f.close()

dset <HDF5 dataset "coords": shape (3, 4), type "<f4">


In [123]:
print(last_points)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [124]:
last_points = torch.from_numpy(last_points)

In [125]:
last_points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])